<a href="https://colab.research.google.com/github/yoonju977/AI/blob/main/%5BAIST%5D_HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Homework #2*

**이름과 학번을 기재해 주세요**
- Name : ooo
- Student Id : 202512345
- Submission date : 2024 / 00 / 00

In [3]:
# 이름과 학번을 기재해 주세요
NAME = "송윤주"
ID = "201701881"

In [4]:
# NOTE - 해당 셀 수정 금지
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset, DataLoader
import random
import torch.optim as optim
import logging

logger = logging.getLogger(__name__)
file_handler = logging.FileHandler(f'hw2_{ID}_{NAME}.log')
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.INFO)

logger.info('HELLO WORLD!!')

INFO:__main__:HELLO WORLD!!


In [5]:
# NOTE - 해당 셀 수정 금지
def set_seed(seed):
    logger.info("Set seed")
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
SEED = 42
set_seed(SEED)

INFO:__main__:Set seed


In [6]:
# NOTE - 해당 셀 수정 금지
logger.info("Q1")

INFO:__main__:Q1


## Q1.1. 다중 조건부 확률 계산을 위한 함수 구현 (20점)
다음의 확률 이론들에 대한 설명을 참조하여, 어떤 사건 A와 조건들 B,C,...에 대해,  
`The Law of Total Probability`, `Chain rule`, `Bayes' Theorem`을 이용하여 다중 조건부 확률을 계산하는 함수를 구현합니다.

#### 1. Chain Rule (체인 룰)

체인 룰은 여러 사건이 연속적으로 일어나는 확률을 각 사건의 조건부 확률로 분해해 계산할 때 사용됩니다. 체인 룰의 기본 형태는 아래와 같습니다.

- 예를 들어, 두 사건 $A$ 와 $B$ 에 대해 체인 룰은 다음과 같이 나타낼 수 있습니다.
  $$
  P(A, B) = P(B \mid A) \cdot P(A)
  $$

- 여러 사건이 주어진 경우 체인 룰은 다음과 같이 확장됩니다.
  $$
  P(A, B, C, D) = P(D \mid A, B, C) \cdot P(C \mid A, B) \cdot P(B \mid A) \cdot P(A)
  $$

---

#### 2. The Law of Total Probability (전체확률의 법칙)

전확률의 법칙은 사건 $B$ 의 확률을 조건부 확률로 분해하여 구할 수 있는 법칙입니다. 이 법칙은 다음과 같은 수식으로 나타낼 수 있습니다:

- 사건 $A$ 와 사건 $A$ 가 일어나지 않을 확률 $\text{not A}$ 가 주어졌을 때,
  $$
  P(B) = P(B \mid A) \cdot P(A) + P(B \mid \text{not A}) \cdot P(\text{not A})
  $$

[wikipedia - 전체확률의 법칙](https://ko.wikipedia.org/wiki/%EC%A0%84%EC%B2%B4_%ED%99%95%EB%A5%A0%EC%9D%98_%EB%B2%95%EC%B9%99)

---

#### 3. Bayes' Theorem (베이즈 정리)
베이즈 정리는 조건부 확률을 뒤집어 계산할 때 사용됩니다. 사건 $B$ 가 주어졌을 때 사건 $A$ 가 발생할 확률을 구할 수 있으며, 전체확률의 법칙을 바탕으로 계산됩니다.

$$
P(A \mid B) = \frac{P(B \mid A) \cdot P(A)}{P(B)}
$$

$$
{\frac {P(B|A_{1})P(A_{1})}{P(B)}} =
{\frac {P(B|A_{1})P(A_{1})}{P(B|A_{1})P(A_{1})+P(B|A_{2})P(A_{2})}}
$$

[wikipedia - 베이즈 정리](https://ko.wikipedia.org/wiki/%EB%B2%A0%EC%9D%B4%EC%A6%88_%EC%A0%95%EB%A6%AC)

---


### 4. conditional probability with multiple conditions (다중 조건부 확률)

다중 조건부 확률은 사건 $A$ 가 여러 조건들 $B, C, D, ...$ 아래에서 발생할 확률을 의미합니다.   
예를 들어, $A$ 가 $B$ 와 $C$ 가 동시에 일어났을 때 발생할 확률 $P(A \mid B, C)$ 을 계산하는 것이 다중 조건부 확률 계산입니다.   
이때 각 조건들이 순차적으로 주어졌다고 생각하고 체인 룰을 통해 확률을 분해하여 계산할 수 있습니다.  

1. 먼저 $P(A \mid B, C)$ 와 같은 확률을 구하려고 할 때, 체인 룰을 이용하여 이를 분해할 수 있습니다.
   $$
   P(A \mid B, C) = \frac{P(A, B, C)}{P(B, C)}
   $$

2. 체인 룰을 이용해 $P(A, B, C)$ 를 계산합니다.

   $$
   P(A, B, C) = P(A \mid B, C) \cdot P(B \mid C) \cdot P(C)
   $$

3. 이렇게 계산한 확률들을 통해 전체 확률을 구하고, 다중 조건부 확률을 계산할 수 있습니다.

---


In [7]:
def chain_rule(probabilities):
    """
    TODO - 체인 룰을 사용하여 다중 조건부 확률을 계산
    - probabilities: 조건부 확률들의 리스트
    예: [P(A|B), P(B|C), P(C|D), ...]
    """
    joint_probability = 1.0
    for prob in probabilities:
        joint_probability *= prob
    return joint_probability

def law_of_total_probability(prior_A, likelihood_conditions_given_A, likelihood_conditions_given_not_A):
    """
    TODO - 전체 확률의 법칙을 사용하여 P(B, C, ...)를 계산
    - prior_A: 사건 A의 사전 확률
    - likelihood_conditions_given_A: 사건 A 하에서의 조건부 확률 리스트 [P(B|A), P(C|B, A), ...]
    - likelihood_conditions_given_not_A: 사건 A' 하에서의 조건부 확률 리스트 [P(B|A'), P(C|B, A'), ...]
    """
    # P(B, C, ... | A) 계산
    joint_given_A = chain_rule(likelihood_conditions_given_A)
    # P(B, C, ... | A') 계산
    joint_given_not_A = chain_rule(likelihood_conditions_given_not_A)
    # 전체 확률의 법칙 적용
    joint_likelihood = prior_A * joint_given_A + (1 - prior_A) * joint_given_not_A
    return joint_likelihood

def bayes_theorem(prior_A, likelihood_conditions_given_A, joint_likelihood_conditions):
    """
    TODO - 베이즈 정리를 통해 다중 조건 하에서 조건부 확률을 계산
    - prior_A: 사건 A의 사전 확률
    - likelihood_conditions_given_A: 사건 A 하에서 조건부 확률 리스트 [P(B|A), P(C|B), ...]
    - joint_likelihood_conditions: 다중 조건의 결합 확률 P(B, C, ...)
    """
    # 체인 룰을 이용해 P(B, C, ... | A)를 계산
    joint_given_A = chain_rule(likelihood_conditions_given_A)

    # 베이즈 정리를 적용해 조건부 확률 계산
    posterior = (joint_given_A * prior_A) / joint_likelihood_conditions
    return posterior

## Q 1.2. (15점)
어떤 병에 걸릴 확률은 5%입니다. 특정 병이 있는 경우 증상 A, 증상 B, 증상 C가 각각 나타날 확률은 다음과 같습니다:

- 질병이 있을 때 : `P(A∣질병)=0.8`,  `P(B∣A,질병)=0.7`, `P(C∣B,A,질병)=0.6`
- 질병이 없을 때 : `P(A∣질병없음)=0.1`, `P(B∣A,질병없음)=0.2`, `P(C∣B,A,질병없음)=0.3`

다음의 조건을 만족하는 `질병에 걸렸을 확률 : P(질병∣A,B,C)` 를 직접 계산하고 풀이과정을 서술하세요.
* 어떤 확률이론을 '왜'썼는지 구체적으로 서술하셔야 합니다.
* 최종 결과는 소수점 아래 다섯째 자리에서 반올림하여, 넷째 자리까지만 써주시면 됩니다.

## A 1.2.

## Q 1.3. (5점)
Q 1.2.의 문제를 **위에서 구현한 함수를 이용하여** 계산하세요.

In [8]:
# TODO - Q 1.1에서 작성한 함수를 이용하여 구현
prior_disease = 0.05
likelihood_conditions_given_disease = [0.8, 0.7, 0.6]
likelihood_conditions_given_no_disease = [0.1, 0.2, 0.3]
# 전체 확률의 법칙을 통해 P(A, B, C) 계산
joint_likelihood_conditions = law_of_total_probability(prior_disease, likelihood_conditions_given_disease, likelihood_conditions_given_no_disease)
# 베이즈 정리를 통해 P(질병|A, B, C) 계산
posterior = bayes_theorem(prior_disease, likelihood_conditions_given_disease, joint_likelihood_conditions)

print(f"P(질병|A, B, C) : {posterior:.4f}")

P(질병|A, B, C) : 0.7467


In [ ]:
# NOTE - 해당 셀 수정 금지
logger.info("Q2")

## Q 2.1. Simple GPT 모델 구현 (40점)
다음의 class 들의 빈칸을 채워 최종적으로 Simplified Generative Pre-trained Transformer (GPT) 모델을 구현하세요.  

코드의 빈칸은 Q1부터 Q15까지 총 15개입니다.

In [9]:
# NOTE - Positional Encoding, 위치 인코딩을 통해 시퀀스의 위치 정보를 제공함
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        # NOTE - d_model은 트랜스포머 모델의 임베딩 차원 수
        super(PositionalEncoding, self).__init__()
        # TODO - Q1 : 위치 인코딩을 저장할 행렬 생성
        positional_embedding_matrix = torch.zeros(max_len, d_model)
        # TODO - Q2 : 위치 벡터 생성
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # 각 차원에 대한 주기적인 함수 적용을 위한 스케일링 값 계산
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        # TODO - Q3 : 짝수 인덱스에는 사인 함수 적용
        positional_embedding_matrix[:, 0::2] = torch.sin(position * div_term)
        # TODO - Q3 : 홀수 인덱스에는 코사인 함수 적용
        positional_embedding_matrix[:, 1::2] = torch.cos(position * div_term)
        # 배치 차원을 추가하여 모양 맞춤
        positional_embedding_matrix = positional_embedding_matrix.unsqueeze(0)
        # 모델이 학습하지 않도록 고정된 상태로 위치 인코딩 저장
        self.register_buffer('positional_embedding_matrix', positional_embedding_matrix)

    def forward(self, x):
        # TODO - Q4 :입력 텐서에 위치 인코딩 더하기
        x = x + self.positional_embedding_matrix[:, :x.size(1)]
        return x


In [17]:
# NOTE - Multi-Head Attention, 멀티 헤드 어텐션을 통해 여러 어텐션을 병렬로 수행하여 더 풍부한 표현 학습
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        # NOTE - d_model은 트랜스포머 모델의 임베딩 차원 수
        # NOTE - num_heads는 어텐션 헤드의 수
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.d_model = d_model
        self.num_heads = num_heads
        # TODO - Q4 : self.d_k는 멀티헤드 어텐션에서 각 어텐션 헤드의 차원 수
        self.d_k = d_model // num_heads

        # TODO - Q5 : 가중치 행렬 초기화
        self.W_q = nn.Linear(d_model, d_model)  # Query 변환
        self.W_k = nn.Linear(d_model, d_model)  # Key 변환
        self.W_v = nn.Linear(d_model, d_model)  # Value 변환
        self.W_o = nn.Linear(d_model, d_model)  # 출력 변환

    def forward(self, Q, K, V, mask=None):
        batch_size = Q.size(0)

        # 1. 선형 변환과 헤드 분할
        # TODO - Q6 : (batch_size, seq_length, d_model) -> (batch_size, num_heads, seq_length, d_k)
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # TODO - Q7 :2. Scaled Dot-product 어텐션 스코어 계산
        scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(self.d_k, dtype=torch.float))

        # 3. 마스킹 적용 (필요한 경우)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # 4. 소프트맥스 적용하여 어텐션 가중치 계산
        attention = F.softmax(scores, dim=-1)

        # TODO - Q8 : 5. Value와 가중치를 곱하여 최종 출력 계산
        out = torch.matmul(attention, V)
        # 6. 헤드 연결과 원래 형태로 변환
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.W_o(out)


In [18]:
# NOTE - Transformer Decoder Block, 트랜스포머 디코더 블록, 멀티 헤드 어텐션과 피드 포워드 네트워크로 구성됨
class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads):
        super(TransformerBlock, self).__init__()
        # 멀티 헤드 어텐션 레이어
        self.attention = MultiHeadAttention(d_model, num_heads)
        # TODO - Q9 : 레이어 정규화
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        # TODO - Q10 : 피드 포워드 네트워크
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_model * 4),  # 확장
            nn.ReLU(),
            nn.Linear(d_model * 4, d_model)   # 압축
        )

    def forward(self, x, mask=None):
        # 1. 멀티 헤드 어텐션 + 잔차 연결과 정규화
        attention = self.attention(x, x, x, mask)
        x = self.norm1(x + attention)
        # TODO - Q11 : 2. 피드 포워드 + 잔차 연결과 정규화
        ff = self.feed_forward(x)
        x = self.norm2(x + ff)
        return x


In [19]:
# GPT 단순화 버전
class GPT(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, d_ff, num_layers, max_len):
        super(GPT, self).__init__()
        # TODO - Q12 : 입력 토큰에 대한 임베딩
        self.embedding = nn.Embedding(vocab_size, d_model)
        # 위치 인코딩 추가
        self.positional_encoding = PositionalEncoding(d_model, max_len)
        # 단순화된 트랜스포머 디코더 블록 (num_layers 개 사용)
        self.layers = nn.ModuleList([TransformerBlock(d_model, num_heads) for _ in range(num_layers)])
        # TODO - Q13 : 출력층 정의 (다음 단어 예측)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # 임베딩과 위치 인코딩을 입력에 적용
        x = self.embedding(x)
        x = self.positional_encoding(x)
        # 트랜스포머 블록 통과시킴
        for layer in self.layers:
            x = layer(x)
        # 마지막으로 출력층을 통해 다음 단어에 대한 로짓 계산
        logits = self.fc_out(x)
        return logits


In [20]:
# HACK - Hyperparameters, 모델의 임의의 하이퍼파라미터 설정
vocab_size = 1000  # 임의의 vocab size
d_model = 128
num_heads = 4  # num_heads 값을 설정
d_ff = 256  # 피드 포워드 네트워크의 차원 수 설정
num_layers = 1  # 레이어 수
max_len = 50
epochs = 70
learning_rate = 0.001  # 학습률 설정

In [21]:
# HACK - 데이터 예시 (간단한 토큰 시퀀스)
data = [
    random.sample(range(vocab_size), 10)
    for _ in range(1000)
    ]

# 데이터셋 준비
# 입력 토큰 시퀀스를 텐서로 변환
tokens = torch.tensor(data, dtype=torch.long)
# 라벨도 동일하게 입력과 같은 데이터로 사용 (다음 단어 예측을 위해)
labels = torch.tensor(data, dtype=torch.long)

# 모델 초기화
model = GPT(vocab_size, d_model, num_heads, d_ff, num_layers, max_len)

In [22]:
# 손실 함수 정의 (크로스 엔트로피 손실)
criterion = nn.CrossEntropyLoss()
# 옵티마이저 정의 (Adam 옵티마이저 사용)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 훈련
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()  # 기울기 초기화
    output = model(tokens)  # 모델에 입력 데이터 전달
    # 손실 계산 (출력과 라벨을 비교)
    loss = criterion(output.view(-1, vocab_size), labels.view(-1))
    # TODO - Q14 : 역전파를 통해 기울기 계산
    loss.backward()
    optimizer.step()  # 옵티마이저로 파라미터 업데이트
    if epoch%10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

print("Training finished!")


Epoch [1/70], Loss: 7.0693
Epoch [11/70], Loss: 5.5995
Epoch [21/70], Loss: 4.0732
Epoch [31/70], Loss: 2.6954
Epoch [41/70], Loss: 1.5389
Epoch [51/70], Loss: 0.7374
Epoch [61/70], Loss: 0.3282
Training finished!


In [25]:
# 텍스트 생성 예시
vocab = {str(i): i for i in range(vocab_size)}
vocab_reverse = {i: str(i) for i in range(vocab_size)}

# NOTE - 간단한 텍스트 생성 함수 호출
def generate_text(model, start_text, vocab, vocab_reverse, max_length=100, temperature=1.0):
    """텍스트 생성 함수"""
    model.eval()
    current_text = start_text
    # 시작 텍스트를 인코딩
    encoded = torch.tensor([[vocab[char] for char in start_text.split()]], dtype=torch.long)

    with torch.no_grad():
        for _ in range(max_length):
            # 현재 시퀀스에 대한 예측
            output = model(encoded)
            # temperature sampling을 위한 logits 조정
            logits = output[0, -1] / temperature
            # 확률 분포 계산
            probs = F.softmax(logits, dim=-1)
            # 다음 토큰 샘플링
            next_char_idx = torch.multinomial(probs, 1, replacement=True).item()
            # TODO - Q15 : 생성된 텍스트에 새 문자 추가
            current_text += " " + vocab_reverse[next_char_idx]
            # 입력 시퀀스 업데이트
            encoded = torch.cat([encoded[:, 1:], torch.tensor([[next_char_idx]])], dim=1)

    return current_text


In [26]:
# NOTE - 랜덤 출력 발생, 토큰 생성여부만 판단
start_text = "1 5 8 3 3"
generated_text = generate_text(model, start_text, vocab, vocab_reverse, max_length=30, temperature=1.0)
print("Generated text : ", generated_text)

Generated text :  1 5 8 3 3 3 3 422 422 47 629 629 629 629 629 629 629 932 932 932 932 932 932 932 932 932 932 932 615 615 615 615 548 548 548


## Q 2.2. (10점)
MultiHeadAttention 클래스에서 사용되는 Q(Query), K(Key), V(Value)의 역할을 각각 설명하세요.   
특히, 각 행렬이 멀티 헤드 어텐션에서 입력 간의 관계를 계산하는 데 어떻게 기여하는지 서술해야 합니다.

## A 2.2.
1.Query (Q):
Query는 현재 입력에 대한 질문 또는 관심의 초점을 나타냅니다.
입력 시퀀스의 각 위치에서, 다른 위치들과의 유사도를 계산하기 위해 사용됩니다.
MultiHeadAttention에서, Query 행렬은 입력 텐서를  W_q 라는 가중치 행렬로 변환하여 생성됩니다.

2.Key (K):
Key는 입력 시퀀스의 각 위치가 “어떤 정보”를 제공할 수 있는지 나타냅니다.
Query와 Key 간의 내적(dot product)을 통해 입력 시퀀스 간의 유사성을 계산하며, 이를 통해 어느 위치가 현재 Query에 얼마나 중요한지 결정합니다.
Key 행렬은 입력 텐서를  W_k 라는 가중치 행렬로 변환하여 생성됩니다.

3.Value (V):
Value는 실제로 전달하고자 하는 정보를 나타냅니다.
Query와 Key를 통해 계산된 가중치(attention scores)를 Value에 적용하여 최종적으로 중요한정보를 강조하거나 덜 중요하게 만듭니다.
Value 행렬은 입력 텐서를  W_v 라는 가중치 행렬로 변환하여 생성됩니다.

  Q, K, V의 관계:
  Query와 Key 간의 내적은 각 위치의 유사도를 측정하여 “어텐션 스코어”를 생성합니다.
  어텐션 스코어를 소프트맥스를 통해 확률 값으로 변환한 후, Value에 곱하여 특정 위치의 정보가 얼마나 중요한지를 반영합니다.
  이를 통해 모델은 입력 데이터의 특정 위치 간 관계를 학습합니다.

## Q 2.3. (10점)
MultiHeadAttention에서 여러 개의 어텐션 헤드를 사용하는 이유와, 각 어텐션 헤드가 Q, K, V를 통해 서로 다른 관계를 학습할 수 있는 이유를 설명하세요.

## A 2.3.
	1.여러 어텐션 헤드를 사용하는 이유:
	MultiHeadAttention은 하나의 어텐션 메커니즘으로 학습할 수 있는 관계가 제한적일 수 있다는 문제를 해결합니다.
	여러 어텐션 헤드를 사용하면 서로 다른 서브 공간(subspace)에서 Query, Key, Value를 독립적으로 학습할 수 있으므로, 다양한 관계와 패턴을 병렬로 학습할 수 있습니다.
	예를 들어, 한 어텐션 헤드는 입력 시퀀스의 문법적 관계를 학습하고, 다른 어텐션 헤드는 문맥적 관계를 학습할 수 있습니다.
  
	2.각 어텐션 헤드가 Q, K, V를 통해 학습하는 관계:
	각 어텐션 헤드는 고유한 가중치 행렬  W_q, W_k, W_v 를 갖고 있어 입력 데이터를 서로 다른 방식으로 변환합니다.
	이를 통해 동일한 입력 데이터라도 각 헤드가 다른 관계를 학습할 수 있습니다.
	예를 들어, 하나의 헤드는 문장 내 단어 간의 짧은 거리 관계를 학습하고, 다른 헤드는 장거리 의존성을 학습할 수 있습니다.
	다양한 헤드가 독립적으로 학습한 정보를 병합(Concatenate)함으로써 더 풍부하고 다양한 표현을 얻을 수 있습니다.

## Bonus (1점)
과제에 대한 피드백을 남겨주세요!

수고하셨습니다 :)  
제출 전 셀을 모두 실행 후 제출 바랍니다!

In [27]:
# NOTE - 해당 셀 수정 금지
logger.info("END")

INFO:__main__:END
